In [2]:
import pandas as pd
import numpy as np
import warnings
from sklearn.datasets import make_classification
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
#from lightgbm import LGBMClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

warnings.filterwarnings(action='ignore')

In [3]:
df = pd.read_csv("modified_train_set.csv")
x_train = df.drop(columns=['loan_status', 'addr_state'])
y_train = df['loan_status']

FileNotFoundError: [Errno 2] No such file or directory: 'modified_train_set.csv'

In [82]:
nan_indices = np.isnan(x_train).any(axis=1)
x_train = x_train[~nan_indices]
y_train = y_train[~nan_indices]

In [83]:
x_train = x_train.astype('float32')
X_Train, X_Validation, Y_Train, Y_Validation = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)

In [84]:
#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state=42)
#X_train_over, y_train_over = sm.fit_resample(X_Train, Y_Train)

In [85]:
from tensorflow.keras.utils import to_categorical
Y_Train=to_categorical(Y_Train, 2).astype(int)
Y_Validation=to_categorical(Y_Validation, 2).astype(int)

In [91]:
from keras import backend as K

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [92]:
# model construct
model = keras.Sequential([
    layers.Dense(60, input_dim=91,activation='relu'),
    layers.Dense(30,activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(2,activation='softmax'),
])

# Define the SGD optimizer
optimizer = keras.optimizers.Adam(learning_rate=0.01)

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy", f1_score, recall, precision])
# Train the model
epochs = 100
batch_size = 200000
early_stopping = EarlyStopping(min_delta=0.01)
model.fit(X_Train, Y_Train, batch_size=batch_size, epochs=epochs)
# Evaluate the model
test_loss, test_acc, f1, re, pre = model.evaluate(X_Validation, Y_Validation, callbacks=[early_stopping])
print("Test accuracy:", test_acc, f1, re, pre)

Epoch 1/100
5/5 [==============================] - 2s 139ms/step - loss: 43332.0391 - accuracy: 0.5793 - f1_score: 0.6068 - recall: 0.6068 - precision: 0.6068
Epoch 2/100
5/5 [==============================] - 1s 118ms/step - loss: 19127.3770 - accuracy: 0.7814 - f1_score: 0.7729 - recall: 0.7729 - precision: 0.7729
Epoch 3/100
5/5 [==============================] - 1s 101ms/step - loss: 7596.5742 - accuracy: 0.5730 - f1_score: 0.5672 - recall: 0.5672 - precision: 0.5672
Epoch 4/100
5/5 [==============================] - 1s 98ms/step - loss: 2825.8853 - accuracy: 0.6695 - f1_score: 0.6795 - recall: 0.6795 - precision: 0.6795
Epoch 5/100
5/5 [==============================] - 1s 102ms/step - loss: 1334.8931 - accuracy: 0.7293 - f1_score: 0.7313 - recall: 0.7313 - precision: 0.7313
Epoch 6/100
5/5 [==============================] - 1s 105ms/step - loss: 385.8685 - accuracy: 0.7901 - f1_score: 0.7893 - recall: 0.7893 - precision: 0.7893
Epoch 7/100
5/5 [==============================] - 1